In [ ]:
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_regression
from sklearn.metrics import r2_score
from sklearn.svm import LinearSVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)

In [43]:
import sys
sys.setrecursionlimit(10000)

### Link to reference: [Link](https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-perform-multioutput-regression-with-svms-in-python.mdm)

In [44]:
path = Path("..")

file = next(path.rglob("**/LAEI_2019_NA_FILLED_WITH_MEAN.csv"))
dataframe = pd.read_csv(file)
print("Data with Mean Imputation:\n", dataframe.head())


print(dataframe.shape)

Data with Mean Imputation:
    Year              Sector          nox         n2o         pm10  \
0  2013    Accidental Fires    21.129667   56.586259    84.125344   
1  2013         Agriculture   244.437997  204.042963    29.984091   
2  2013            Aviation  3851.256755   56.586259    69.838248   
3  2013             Biomass  1586.319624   56.586259  1054.000324   
4  2013  Commercial Cooking  1586.319624   56.586259   547.828374   

         pm2.5           co2  
0    78.041130  9.550853e+05  
1    16.049516  6.776104e+03  
2    58.381094  1.054197e+06  
3  1054.000324  9.550853e+05  
4   547.828374  9.550853e+05  
(48, 7)


In [19]:
dataframe['Year'].unique()

array([2013, 2016, 2019])

In [21]:
dataframe.head()

,Year,Sector,nox,n2o,pm10,pm2.5,co2
0,2013,Accidental Fires,21.129667,56.586259,84.125344,78.041130,9.550853e+05
1,2013,Agriculture,244.437997,204.042963,29.984091,16.049516,6.776104e+03
2,2013,Aviation,3851.256755,56.586259,69.838248,58.381094,1.054197e+06
3,2013,Biomass,1586.319624,56.586259,1054.000324,1054.000324,9.550853e+05
4,2013,Commercial Cooking,1586.319624,56.586259,547.828374,547.828374,9.550853e+05


In [22]:
# filter training data
dataframe = dataframe[dataframe['Year'].isin([2013, 2016, 2019, 2025])]
dataframe.shape

(48, 7)

In [23]:
# proportion of missing values per column
dataframe.isnull().mean()

Year      0.0
Sector    0.0
nox       0.0
n2o       0.0
pm10      0.0
pm2.5     0.0
co2       0.0
dtype: float64

In [24]:
# fill missing values with the median
columns_with_nulls = dataframe.columns[dataframe.isnull().any()].tolist()
# columns_with_nulls

In [25]:
# Replace nulls with the median of specified columns per 'Grid ID 2019'
for column in columns_with_nulls:
    dataframe[column] = dataframe.groupby('Grid ID 2019')[column].transform(lambda x: x.fillna(x.median()))

# train_dataframe.isnull().sum()

In [9]:
dataframe.head()

,Year,Sector,nox,n2o,pm10,pm2.5,co2
0,2013,Accidental FiresAgricultureAviationBiomassComm...,67998.611444,2007.135045,9877.244675,4834.691018,3.676668e+07
1,2016,Accidental FiresAgricultureAviationBiomassComm...,62880.177309,2179.036897,9770.843908,4368.504513,3.874149e+07
2,2019,Accidental FiresAgricultureAviationBiomassComm...,52953.303739,2518.371060,9389.845447,4110.513467,3.712463e+07
3,2025,Accidental FiresAgricultureAviationBiomassComm...,38298.716117,2284.059300,8726.431828,3635.249158,3.571314e+07
4,2030,Accidental FiresAgricultureAviationBiomassComm...,32054.313404,2287.065063,8255.946953,3369.858919,3.261456e+07


In [47]:
# remove ID columns
# dataframe = dataframe.drop(columns=['Grid ID 2019', 'LAEI 1km2 ID', 'Emissions Unit'])
# dataframe.head()

(64, 7)

## 2. Data clean up and handling missing values

In [26]:
# convert easting and northing to categorical
dataframe['Sector'] = dataframe['Sector'].astype('category')


In [27]:
### One-Hot Encoding of Categorical Variables

dataframe_encoded = pd.get_dummies(dataframe, drop_first=True)
dataframe_encoded.head()

,Year,nox,n2o,pm10,pm2.5,co2,Sector_Agriculture,Sector_Aviation,Sector_Biomass,Sector_Commercial Cooking,Sector_Construction,Sector_Forestry,Sector_Gas Leakage,Sector_Heat and Power Generation,Sector_Industrial Processes,Sector_Machinery,Sector_Rail,Sector_Resuspension,Sector_River,Sector_Road Transport,Sector_Waste
0,2013,21.129667,56.586259,84.125344,78.041130,9.550853e+05,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,2013,244.437997,204.042963,29.984091,16.049516,6.776104e+03,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,2013,3851.256755,56.586259,69.838248,58.381094,1.054197e+06,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
3,2013,1586.319624,56.586259,1054.000324,1054.000324,9.550853e+05,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
4,2013,1586.319624,56.586259,547.828374,547.828374,9.550853e+05,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False


## 4. Train a model

### 4.1 Train test split

In [30]:
pollutants = ["nox", "pm10", "pm2.5", "co2"]
train_years = [2013, 2016, 2019]
test_years = [2019]

X = dataframe_encoded.drop(columns=pollutants)
Y = dataframe_encoded[pollutants + ['Year']]

X_train = X[X['Year'].isin(train_years)].drop(columns='Year')
Y_train = Y[Y['Year'].isin(train_years)].drop(columns='Year')

X_test = X[X['Year'].isin(test_years)].drop(columns='Year')
Y_test = Y[Y['Year'].isin(test_years)].drop(columns='Year')

In [31]:
# scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### 4.2 Train the model

In [51]:
# Create the SVR regressor
# svr = SVR(epsilon=0.2)

In [32]:
# wrap the SVR in MultiOutputRegressor
# mor = MultiOutputRegressor(LinearSVR(max_iter=10000))
mor = MultiOutputRegressor(SVR())

In [33]:
# train the regressor
mor.fit(X_train_scaled, Y_train)

MultiOutputRegressor(estimator=SVR())

In [34]:
# average r2 score for all pollutants
mor.score(X_train_scaled, Y_train)

-0.19645498968871067

In [35]:
Y_pred = mor.predict(X_test_scaled)

In [36]:
# r2 score for each pollutant
Y_pred = mor.predict(X_train_scaled)
# convert to df
Y_pred = pd.DataFrame(Y_pred, columns=pollutants)

r2_scores = {col: r2_score(Y_train[col], Y_pred[col]) for col in Y_test.columns}

print('\n')
print('-'*50)

print('MAE nox:', mean_absolute_error(Y_test['nox'], Y_pred[:, 0]))
print('MAE pm10:', mean_absolute_error(Y_test['pm10'], Y_pred[:, 1]))
print('MAE pm2.5:', mean_absolute_error(Y_test['pm2.5'], Y_pred[:, 2]))
print('MAE co2:', mean_absolute_error(Y_test['co2'], Y_pred[:, 3]))


MSE nox: 42645164.535078585
MSE pm10: 988146.732216735
MSE pm2.5: 168412.11585217173
MSE co2: 21456813235197.637


--------------------------------------------------
MAE nox: 3396.156498127555
MAE pm10: 550.7769262457467
MAE pm2.5: 222.06479743787398
MAE co2: 2353711.539181128


In [37]:
# -------

In [46]:
### Process Categorical Variables
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Initialize the Label Encoder
encoder = LabelEncoder()

# Encode 'Main Source Category' using explicit row and column indexing
# Encode 'Main Source Category' using explicit row and column indexing
dataframe["Sector"] = encoder.fit_transform(dataframe.loc[:, "Sector"])

# Trying to use date and sector as feature to predict the pollutants
model = SVC()
X = dataframe.iloc[:, :2]
y = dataframe.iloc[:, 2:]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=15)
# model.fit(X_train, y_train)
# model.score(X_test, y_test)

# # X_test[[]]model.predict(X_test)
# X_test[['nox_pred', 'n2o_pred', 'pm10_pred', 'pm2.5_pred', 'co2_pred']] = model.predict(X_test)


In [47]:
X_train

,Year,Sector
30,2016,14
35,2019,3
43,2019,11
47,2019,15
42,2019,10
20,2016,4
40,2019,8
2,2013,2
44,2019,12
1,2013,1
